# Netflix shows example

https://www.kaggle.com/shivamb/netflix-shows


In [1]:
# To find out where the pyspark
import findspark
findspark.init()

In [2]:
# Creating Spark Context
from pyspark import SparkContext
sc = SparkContext("local[4]", "netflix")

https://towardsdatascience.com/pyspark-import-any-data-f2856cda45fd

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [4]:
df = spark.read.csv("netflix_titles.csv", header =True, inferSchema=True)
df.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [5]:
#df.show(5)

In [6]:
from pyspark.sql.types import *

schema = StructType([
    StructField("show_id", StringType()),
    StructField("type", StringType()),
    StructField("title", StringType()),
    StructField("director", StringType()),
    StructField("cast", StringType()),
    StructField("country", StringType()),
    StructField("date_added", StringType()),
    StructField("release_year", IntegerType()),
    StructField("rating", StringType()),
    StructField("duration", StringType()),
    StructField("listed_in", StringType()),
    StructField("description", StringType())])
df = spark.read.csv("netflix_titles.csv", header = 'true', schema=schema)
df.printSchema()
#df.show(5)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [7]:
print("Count: %d" % df.count())

Count: 6236


In [8]:
df.select(["show_id", "date_added"]).show(5)

+--------+-----------------+
| show_id|       date_added|
+--------+-----------------+
|81145628|September 9, 2019|
|80117401|September 9, 2016|
|70234439|September 8, 2018|
|80058654|September 8, 2018|
|80125979|September 8, 2017|
+--------+-----------------+
only showing top 5 rows



### Fix datetime

In [9]:
import datetime

In [13]:
# Spark 3 changed date format https://kb.databricks.com/en_US/sql/using-datetime-values-in-spark-30-and-above
# We apply some conversion first
from pyspark.sql import functions as F
df_new = df.withColumn("date_added_v2", F.translate(df['date_added'], ",", ""))
df_new.select(['show_id', 'date_added', 'date_added_v2']).show(5)

df2 = df_new.withColumn("show_date",F.expr("to_date(date_added_v2, 'MMMM d y')"))
df2.select(['show_id', 'date_added', 'date_added_v2', 'show_date']).show(5)


+--------+-----------------+----------------+
| show_id|       date_added|   date_added_v2|
+--------+-----------------+----------------+
|81145628|September 9, 2019|September 9 2019|
|80117401|September 9, 2016|September 9 2016|
|70234439|September 8, 2018|September 8 2018|
|80058654|September 8, 2018|September 8 2018|
|80125979|September 8, 2017|September 8 2017|
+--------+-----------------+----------------+
only showing top 5 rows

+--------+-----------------+----------------+----------+
| show_id|       date_added|   date_added_v2| show_date|
+--------+-----------------+----------------+----------+
|81145628|September 9, 2019|September 9 2019|2019-09-09|
|80117401|September 9, 2016|September 9 2016|2016-09-09|
|70234439|September 8, 2018|September 8 2018|2018-09-08|
|80058654|September 8, 2018|September 8 2018|2018-09-08|
|80125979|September 8, 2017|September 8 2017|2017-09-08|
+--------+-----------------+----------------+----------+
only showing top 5 rows



In [14]:
#import pyspark.sql.functions as F
#
#
#df2 = df.withColumn("show_date",F.expr("to_date('September 12, 2005', 'MMMM dd, yyyy')"))
#df2 = df.withColumn("show_date",F.expr("to_date(date_added, 'MMMM dd, yyyy')"))
#
#df2.select(["show_id", "date_added", "show_date"]).show(5)


In [15]:
df2.select(["show_id", "date_added", "show_date"]).show(5)

+--------+-----------------+----------+
| show_id|       date_added| show_date|
+--------+-----------------+----------+
|81145628|September 9, 2019|2019-09-09|
|80117401|September 9, 2016|2016-09-09|
|70234439|September 8, 2018|2018-09-08|
|80058654|September 8, 2018|2018-09-08|
|80125979|September 8, 2017|2017-09-08|
+--------+-----------------+----------+
only showing top 5 rows



In [16]:
dates = ("2018-01-01",  "2018-12-12")
date_from, date_to = [F.to_date(F.lit(s)).cast(TimestampType()) for s in dates]

df2.where((df2.show_date > date_from) & (df2.show_date < date_to)).select("show_id", "title", "show_date").show(5)

+--------+--------------------+----------+
| show_id|               title| show_date|
+--------+--------------------+----------+
|70234439|  Transformers Prime|2018-09-08|
|80058654|Transformers: Rob...|2018-09-08|
|80244601|     Castle of Stars|2018-09-07|
|80203094|         City of Joy|2018-09-07|
|80190843|      First and Last|2018-09-07|
+--------+--------------------+----------+
only showing top 5 rows



In [17]:
df2.select("country").distinct().sort(F.asc("country")).show()


+--------------------+
|             country|
+--------------------+
|                null|
|     Ama K. Abebrese|
|         Aziz Ansari|
|       Dominic Costa|
|          Doug Plaut|
| Justin ""Alyssa ...|
|  Lachion Buckingham|
|          Rob Morgan|
|                1944|
|           Argentina|
|Argentina, Brazil...|
|    Argentina, Chile|
|Argentina, Chile,...|
|   Argentina, France|
|Argentina, France...|
|    Argentina, Italy|
|    Argentina, Spain|
|Argentina, United...|
|Argentina, United...|
|Argentina, Urugua...|
+--------------------+
only showing top 20 rows



In [18]:
df2.where(F.col("country").isNull()).count()

478

In [19]:
df3 = df2.where(F.col("country").isNotNull())

In [20]:
df3.count()

5758

In [21]:
df3.select("country").distinct().sort(F.asc("country")).show()


+--------------------+
|             country|
+--------------------+
|     Ama K. Abebrese|
|         Aziz Ansari|
|       Dominic Costa|
|          Doug Plaut|
| Justin ""Alyssa ...|
|  Lachion Buckingham|
|          Rob Morgan|
|                1944|
|           Argentina|
|Argentina, Brazil...|
|    Argentina, Chile|
|Argentina, Chile,...|
|   Argentina, France|
|Argentina, France...|
|    Argentina, Italy|
|    Argentina, Spain|
|Argentina, United...|
|Argentina, United...|
|Argentina, Urugua...|
|Argentina, Urugua...|
+--------------------+
only showing top 20 rows



In [22]:
split_col = F.split(df['country'], ',')
df4 = df3.withColumn('main_country', split_col.getItem(0))

In [23]:
df4.select("country", "main_country").show(5)


+--------------------+--------------+
|             country|  main_country|
+--------------------+--------------+
|United States, In...| United States|
|      United Kingdom|United Kingdom|
|       United States| United States|
|       United States| United States|
|       United States| United States|
+--------------------+--------------+
only showing top 5 rows



In [39]:
df4.select("main_country").distinct().sort(F.asc("main_country")).show(), df4.select("main_country").distinct().count()

+--------------------+
|        main_country|
+--------------------+
|     Ama K. Abebrese|
|         Aziz Ansari|
|       Dominic Costa|
|          Doug Plaut|
| Justin ""Alyssa ...|
|  Lachion Buckingham|
|          Rob Morgan|
|                1944|
|           Argentina|
|           Australia|
|             Austria|
|          Bangladesh|
|             Belgium|
|              Brazil|
|            Bulgaria|
|            Cambodia|
|              Canada|
|               Chile|
|               China|
|            Colombia|
+--------------------+
only showing top 20 rows



(None, 80)

In [37]:
df4.select("main_country").distinct().count()

80

In [25]:
df4.groupby("main_country").count().sort(F.desc("count")).show(30)

+--------------+-----+
|  main_country|count|
+--------------+-----+
| United States| 2294|
|         India|  808|
|United Kingdom|  483|
|        Canada|  206|
|         Japan|  184|
|        France|  147|
|   South Korea|  146|
|         Spain|  139|
|        Mexico|   99|
|     Australia|   94|
|        Turkey|   83|
|         China|   77|
|     Hong Kong|   73|
|        Taiwan|   71|
|       Germany|   65|
|        Brazil|   57|
|     Argentina|   56|
|      Thailand|   53|
|         Egypt|   52|
|     Indonesia|   48|
|   Philippines|   46|
|         Italy|   40|
|       Nigeria|   36|
|      Colombia|   29|
|       Denmark|   28|
|       Ireland|   22|
|      Pakistan|   21|
|        Israel|   20|
|     Singapore|   19|
|   Netherlands|   19|
+--------------+-----+
only showing top 30 rows



In [26]:
df4.select("type").distinct().show()

+-------------+
|         type|
+-------------+
|      TV Show|
|        Movie|
|William Wyler|
+-------------+



In [27]:
df4.groupby("release_year").agg(F.count("type").alias("num_types")).sort(F.col("num_types").desc()).show()

+------------+---------+
|release_year|num_types|
+------------+---------+
|        2018|      953|
|        2017|      899|
|        2016|      765|
|        2019|      725|
|        2015|      478|
|        2014|      272|
|        2013|      225|
|        2012|      170|
|        2010|      138|
|        2011|      133|
|        2009|      114|
|        2008|      103|
|        2007|       68|
|        2006|       68|
|        2005|       62|
|        2004|       48|
|        2003|       41|
|        2002|       38|
|        2001|       33|
|        1997|       31|
+------------+---------+
only showing top 20 rows



In [28]:
 movies = df4.createOrReplaceTempView("movies")

In [29]:
spark.sql("""
select                
release_year,
count(type) AS num_types
from movies
GROUP BY release_year
ORDER BY num_types  DESC
""").show()

+------------+---------+
|release_year|num_types|
+------------+---------+
|        2018|      953|
|        2017|      899|
|        2016|      765|
|        2019|      725|
|        2015|      478|
|        2014|      272|
|        2013|      225|
|        2012|      170|
|        2010|      138|
|        2011|      133|
|        2009|      114|
|        2008|      103|
|        2007|       68|
|        2006|       68|
|        2005|       62|
|        2004|       48|
|        2003|       41|
|        2002|       38|
|        2001|       33|
|        1997|       31|
+------------+---------+
only showing top 20 rows



In [ ]:
sc.stop()